# 第5章　音のフィルタリング（確認問題の解答）
コードのセルを，本文ノートブック Chap5.ipynb にコピペして実行してください。

## 5.1 FIR フィルタによる雑音の除去 
### (a) インパルス応答の畳み込み 


(1) 上記 (b) では，雑音除去のために LPF を用いた。もしプログラム中の

wave_data += noise.astype(int)                     # 雑音を重畳

の行をコメントアウトすれば，「音楽の高域をカットすることによる音質の変化」を観測することができる。カットオフ周波数を様々に変えて，音質の変化を観測しなさい。（ヒント： fc = 8e3 とすればラジオの AM 放送なみ，fc = 3.4e3 とすれば電話音声なみの音質になる）。

【答】電話音声帯域では「音楽を伝送するのには不十分」と感じていただけたでしょうか？

(2) 上記 (b) では，h_final （円状シフトした後，窓掛けしたインパルス応答）を用いて LPF をかけた。これを，h （逆 FFT しただけ）や，h_shifted （円状シフトしただけ）を用いた場合に，h_final を用いた場合と違いがあるか，確認しなさい。

【答】元のノートブックからインプット5.4～5.8をコピーして，整理したコードを以下に示します。本質的な変更はインプット5.8の部分における，h_final を h_shifted や h に置換した部分だけです。

In [ ]:
# インプット5.4
# (1) 遮断周波数を決め，その周波数未満は振幅 1 ，それ以上は振幅 0 のペクトルを用意

fs = 44.1e3  # 標本化周波数は 44.1 kHz
N = 64      # タップ長を設定
fc = 6e3    # 遮断周波数（カットオフ周波数）を設定

kc = int(np.floor(fc / fs * N ))  # 遮断周波数に対応する，周波数ビンを算出
H = np.zeros(N)                   # まずは，全周波数ビンの振幅を 0 として初期化
H[0: kc] = 1.0                    # 遮断周波数未満は振幅を 1 に設定
H[-kc+1: ] = 1.0                  # 負の周波数領域における振幅値も 1 に設定

# インプット5.5
# (2) 逆 FFT することで，インパルス応答を算出
h = np.real( np.fft.ifft(H) )  # 逆 FFT して，虚部のゴミを無視

# インプット 5.6
# (3-A) 円状シフトを実施
h_shifted = circle_shift(h, int(N/2))

# インプット 5.7 
fs, wave_data_original = scipy.io.wavfile.read ('sample/sample3.wav')
sampling_interval = 1.0 / fs
times = np.arange ( len ( wave_data_original )) * sampling_interval
wave_data = wave_data_original.copy()

mod = 2e3*np.cos(2*np.pi*0.25*times)
noise = 500.0 * np.sin(2*np.pi*8e3*times + mod)
wave_data += noise.astype(int)                     # 雑音を重畳します。

# インプット 5.8
# 型を float に変換したうえで，畳み込み
result = np.convolve(h_shifted, wave_data.astype(float))   # h_final を h_shifted と置換
#result = np.convolve(h, wave_data.astype(float))         # h_final を h と置換

times = np.arange ( len ( result )) * sampling_interval
plt.title("Output 5.8")
plot_wave ( times, result )                          # 波形全体を表示
plot_wave ( times[0:1000], result[0:1000] )          # 波形の先頭1000点部分のみ表示

plt.specgram(result[:], NFFT=256, Fs=fs, cmap='jet')
# モノクロプリンタ用には，jet → gray などにしてみるとよい
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()

audio = Audio(result[:], rate = fs)
audio

まず，h_final の代わりに h_shifted を用いると，スペクトログラムの色の変化が急峻であることから，遮断特性は急峻であることが分かります。ただし，波形の先頭部分1000ポイントを拡大して表示すると，入力音の立ち上がりを反映した「暴れ」があることが分かります（同じことを h_final を使った場合についても表示してみれば，違いはかなり大きいことが分かります）。

次に，上記のコードで，インプット5.8 の部分における 「# h_final を h と置換」という注釈がついた行のコメントを外してから再度実行してください。h_final の代わりに h を用いると，LPFとしての機能が弱いことが分かります。円状畳み込みの世界で設計したフィルタを，直線畳み込みしているので，うまくいかないのは当然といえるでしょう。

(3)  [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html)  において，Filter design のうち [firwin の説明](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html#scipy.signal.firwin)  を読み，5 kHz 以下をカットする HPF を設計し，フィルタリングしてみなさい。（ヒント：firwin の説明を完璧に理解できなくても OK です。説明の下の Examples があり，その中の「High-pass (‘stop’ from 0 to f):」と書いてある部分を読めば見当がつきます。ただし，フィルタ長は奇数にする必要があります。）

【答】 インプット5.9と5.10のうち，フィルタ設計の行に  pass_zero=False を追加するだけです。N = 63としました。

In [ ]:
# インプット 5.9
from scipy import signal
N = 63
fc = 5e3 / (fs / 2.0) # 遮断周波数は，ナイキスト周波数 (fs/2) に対する比で指定 
h = signal.firwin(N, fc, pass_zero=False, window='hann') # ここで HPF の設計
plt.title("Output 5.9")
plot_wave([], h)

# インプット 5.10
fs, wave_data_original = scipy.io.wavfile.read ('sample/sample3.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
times = np.arange ( len ( wave_data )) * sampling_interval
wave_data = wave_data_original.copy()

mod = 2e3*np.cos(2*np.pi*0.25*times)
noise = 500.0 * np.sin(2*np.pi*8e3*times + mod)
wave_data += noise.astype(int)                     # 雑音を重畳

result = np.convolve(h, wave_data.astype(float)) # 型を float に変換して畳み込み

times = np.arange ( len ( result )) * sampling_interval
plt.title("Output 5.10")
plot_wave ( times, result )

plt.specgram(result[:], NFFT=256, Fs=fs, cmap='jet')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()

audio = Audio(result[:], rate = fs)
audio

実行結果を聴取すると「音楽のうちの高周波成分」と「周波数変動する雑音」のみが聞こえます。

## 5.2 IIR フィルタによる雑音の除去
(1) 上記 (a) では，$N=16$ の IIR フィルタを設計した。それと同様な遮断特性を得るためには，FIR フィルタでは次数（タップ長）$N$ をどの程度にする必要があるか，試行錯誤的に確認しなさい。

【答】試行錯誤しましょう。まず，$N=16$ の IIR フィルタの遮断特性を確認します。ただし，遮断特性を詳しく見るために，インパルス応答を128点計算して，さらに振幅特性の縦軸を dB 表示にします。また，位相特性は見ないこととします。

In [ ]:
# インプット 5.11
from scipy import signal

N = 16                          # IIR の次数は 16
fc = 6e3 / (fs / 2.0)                             
# 遮断周波数は，ナイキスト周波数 (fs/2) に対する比として指定
b, a = signal.iirfilter(N, fc, btype='lowpass')   
# これで設計（係数 b(k), a(k) の計算）を完了

N_test = 128                    # インパルス応答を 128 点分計算
unit_pulse = np.append(np.array([1.0]), \
                       np.array([0.0 for i in range(N_test - 1)]))    
y = signal.lfilter(b, a, unit_pulse)

plt.subplot(3,1,1)
plt.title("Output 5.11")
plot_wave([],y)

sp = np.fft.fft(y)
plt.subplot(3,1,2)
draw_FFT_spectrum(sp, fs, level=True, draw_range=50.0, phase_spectrum=False)

続いて，FIRフィルタの次数を変えながら特性を見てみましょう。

In [ ]:
# インプット 5.9
from scipy import signal
fc = 6e3 / (fs / 2.0) # 遮断周波数は，ナイキスト周波数 (fs/2) に対する比で指定 

N_candidate = [2**i for i in range(4, 8)]

for N in N_candidate:
    h = signal.firwin(N, fc, window='hann') # 窓関数は「ハニング窓」を利用
    plt.subplot(3,1,1)
    plt.title("N = " + str(N))
    plot_wave([], h)
    result = np.convolve(h, unit_pulse)[0:N_test] # 単位パルスに畳み込み，最初のN_test点だけを取り出す
    sp = np.fft.fft(result)
    plt.subplot(3,1,2)    
    draw_FFT_spectrum(sp, fs, level=True, draw_range=50.0, phase_spectrum=False)

FIRの次数16では，明らかに遮断特性が劣ります。64次くらいで，ほぼ同等でしょうか？これは，IIR（16次）のインパルス応答が，ほぼ70点くらいで収束することと対応しています。

(2)  [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html) において，Filter design のうち [iirfilter の説明](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.iirfilter.html#scipy.signal.iirfilter)を読み，7 kHz ～ 9 kHz の帯域のみを通過させる BPF を設計し，雑音成分だけを取りだしなさい。（ヒント：例題にあるように [低域側遮断周波数, 高域側遮断周波数] のリストを渡すだけです。遮断周波数は，ナイキスト周波数に対する比として与えることに注意してください。）

【答】以下のとおりです。

In [ ]:
# インプット 5.11
from scipy import signal

N = 13                          # IIR の次数は 13
fc = 6e3 / (fs / 2.0)                             
fc_low  = 7e3 / (fs / 2.0)                             
fc_high = 9e3 / (fs / 2.0)     
# 遮断周波数は，ナイキスト周波数 (fs/2) に対する比として指定
b, a = signal.iirfilter(N, [fc_low, fc_high], btype='bandpass')  # これで設計（係数 b(k), a(k) の計算）を完了

# インプット 5.12
fs, wave_data_original = scipy.io.wavfile.read ('sample/sample3.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
times = np.arange ( len ( wave_data )) * sampling_interval
wave_data =  wave_data_original.copy()

mod = 2e3*np.cos(2*np.pi*0.25*times)
noise = 500.0 * np.sin(2*np.pi*8e3*times + mod)
wave_data += noise.astype(int)                     # 雑音を重畳

result = signal.lfilter(b, a, wave_data)

times = np.arange ( len ( result )) * sampling_interval
plt.title("Output 5.12")
plot_wave ( times, result )

plt.specgram(result[:], NFFT=256, Fs=fs, cmap='jet')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()

audio = Audio(result[:], rate = fs)
audio

## 5.3 頭部伝達関数を用いた音像定位の制御
以下の課題を通じて，2次元配列 (ndarray) やイテレータの使い方に習熟すること。

(1) 上記 (b) のプログラムをコピーして，別のモノラルの音楽.wavファイル（例えば TopOfTheWorld.wav）を読み込み，「右45°のスピーカからの音楽を聴取する状況」をヘッドホンで実現しなさい。HRIR データとしては，L0e315a.dat と R0e315a.dat を利用すること。

(2) 元のプログラム，および (1) で作成したプログラムをマージして，「左45°と右45°から違う音楽が聞こえてくる状況」をヘッドホンで実現しなさい。2つの音の長さが違う場合にも対処できるようにすること。

【答】課題 (1) の前に，まず復習です。単にインプット 5.13 ～ 5.15 をコピーして，整理したコードを以下に示します。実行して，左側から聞こえる音を作ります。

In [ ]:
#from read_text_data import * # テキストデータから数値を読み込む関数の読み込み

# インプット 5.13
import sys
import struct
fs = 44100.0
sampling_interval = 1.0 / fs
LR = {"Left":0, "Right":1}               # ディクショナリLR を定義し，左と右に 0 と 1 を割り当て，配列のインデックスとします。

hrir_L = read_text_data('sample/L0e045a.dat')   # データ長が不明なので，二つの配列に分けて読み込みます。
hrir_R = read_text_data('sample/R0e045a.dat')   # もしデータ長が既知ならば，はじめから2次元配列に読み込むべきでしょうね。

hrir_len = min(len(hrir_L), len(hrir_R))
hrir = np.zeros((2, hrir_len))           # 両耳のデータを納める2次元配列を用意します。
hrir[LR["Left"]]  = hrir_L[0: hrir_len]
hrir[LR["Right"]] = hrir_R[0: hrir_len]

times = np.arange(hrir_len) * sampling_interval

print("=== Output 5.13 ===")
for lr in LR:                            # ディクショナリもイテレータです。
    plt.subplot(2, 1, LR[lr]+1)
    plt.title("HRIR (" + lr + " ear)")
    plot_wave(times, hrir[LR[lr]]);      # HRIR の波形を描画する．

# インプット 5.14
fs, wave_data = scipy.io.wavfile.read ('sample/sample.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
times = np.arange(len(wave_data)) * sampling_interval

# インプット 5.15
result = np.zeros((2,len(wave_data) + hrir_len - 1))   
# N点の信号とM点の信号を畳み込んだ結果は N+M-1 点
times = np.arange(result.shape[1]) * sampling_interval 
# 2次元配列 a の列の数は，a.shape[1] で求められます。 

print("=== Output 5.15 ===")
for lr in LR:
    result[LR[lr]] = np.convolve(hrir[LR[lr]], wave_data.astype(float))
    plt.title("Convolved music (" + lr + " ear)")
    plot_wave( times, result[LR[lr]] )

audio = Audio(result, rate = fs)
audio

続いて，課題(1)に対応するために，上記のコードにおいて読み込む HRTF データと音楽データを変更したコードを以下に示します。ただし，課題(2) で再利用するために，出来上がりの音の配列名に2をつけておきます。

In [ ]:
# インプット 5.13
import sys
import struct
fs = 44100.0
sampling_interval = 1.0 / fs
LR = {"Left":0, "Right":1}               # ディクショナリLR を定義し，左と右に 0 と 1 を割り当て，配列のインデックスとします。

hrir_L = read_text_data('sample/L0e315a.dat')   # データ長が不明なので，二つの配列に分けて読み込みます。
hrir_R = read_text_data('sample/R0e315a.dat')   # もしデータ長が既知ならば，はじめから2次元配列に読み込むべきでしょうね。

hrir_len = min(len(hrir_L), len(hrir_R))
hrir = np.zeros((2, hrir_len))           # 両耳のデータを納める2次元配列を用意します。
hrir[LR["Left"]]  = hrir_L[0: hrir_len]
hrir[LR["Right"]] = hrir_R[0: hrir_len]

times = np.arange(hrir_len) * sampling_interval

print("=== Output 5.13 ===")
for lr in LR:                            # ディクショナリもイテレータです。
    plt.subplot(2, 1, LR[lr]+1)
    plt.title("HRIR (" + lr + " ear)")
    plot_wave(times, hrir[LR[lr]]);      # HRIR の波形を描画する．

# インプット 5.14
fs, wave_data = scipy.io.wavfile.read ('sample/sample2.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
times = np.arange(len(wave_data)) * sampling_interval

# インプット 5.15
result2 = np.zeros((2,len(wave_data) + hrir_len - 1))   
# N点の信号とM点の信号を畳み込んだ結果は N+M-1 点
times = np.arange(result2.shape[1]) * sampling_interval 
# 2次元配列 a の列の数は，a.shape[1] で求められます。 

print("=== Output 5.15 ===")
for lr in LR:
    result2[LR[lr]] = np.convolve(hrir[LR[lr]], wave_data.astype(float))
    plt.title("Convolved music (" + lr + " ear)")
    plot_wave( times, result2[LR[lr]] )

audio = Audio(result2, rate = fs)
audio

課題(2) は，これまでに作った2つの音 result と result2 を左右それぞれで重ねる（足し合わせる）だけです。ただし，2つの音の継続時間が異なることには対処が必要です。ここでは，短い方の音に「無音区間」を付加することで長さを揃えることにします。

In [ ]:
# 「無音区間」は np.zeros で作ります。2つの2次元配列は np.concatenate でも結合できますが，ここでは hstack を使ってみます。
if result.shape[1] >= result2.shape[1]:
    result2 = np.hstack([result2, np.zeros((2, result.shape[1] - result2.shape[1]))])
else:
    result  = np.hstack([result,  np.zeros((2, result2.shape[1] - result.shape[1]))])

final = result + result2
audio = Audio(final, rate = fs)
audio

## 5.4 FFT を利用した長い音のフィルタリング

(1) 5.3 節 (b) において実施した「HRTF の畳み込みによる音像定位」は，音楽ファイル全体に対する畳み込みをするものであった。これを，5.4 節 (c) の手順により，ブロック単位でバッファしては FFT を用いて処理する方式に書き換えなさい。

【答】インプット5.21を書き替えてみました。要点は，「モノラルの場合には，appendで伸ばしていく（オーバーラップアドしていく）」のノリでよいのですが，「ステレオの場合には，片チャネルの配列だけを伸ばすことはできない」です。そこで，畳み込み結果を納める2次元h配列を予め準備しておき，「そこから 2N 点を取り出し，N点だけオーバーラップアドして書き戻す」こととしました。

In [ ]:
# インプット 5.21
fs = 44.1e3

# --- フィルタの準備 ＝ HRIRの読み込み（インプット 5.14からコピー）---
fs = 44100.0
sampling_interval = 1.0 / fs
LR = {"Left":0, "Right":1}               # ディクショナリLR を定義し，左と右に 0 と 1 を割り当て，配列のインデックスとします。

hrir_L = read_text_data('sample/L0e045a.dat')   # データ長が不明なので，二つの配列に分けて読み込みます。
hrir_R = read_text_data('sample/R0e045a.dat')   # もしデータ長が既知ならば，はじめから2次元配列に読み込むべきでしょうね。

hrir_len = min(len(hrir_L), len(hrir_R)) # 左右の長い方を採用します。
N = 2**(int(np.log2(hrir_len)) +1 )      # それよりも長い 2 のベキ乗の値を処理の長さとします。

hrir = np.zeros((2, 2 * N))              # 両耳のデータを納める2次元配列を用意します。
hrir[LR["Left"], 0: hrir_len]  = hrir_L[0: hrir_len]
hrir[LR["Right"], 0: hrir_len] = hrir_R[0: hrir_len]

H = np.zeros((2, 2*N), dtype=complex)    # HRTF を納める2次元配列を準備し，
for lr in LR:
    H[LR[lr]] = np.fft.fft(hrir[LR[lr]])

# --- 音の準備 ---
fs, x_original = scipy.io.wavfile.read ('sample/sample3.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
x = x_original.copy()


# --- 処理の開始 ---
n_loop = int(np.floor(len(x) / N))  # 最後の区間長に満たない点は，処理対象から除外

start = 0                                 # 切り出し区間の開始点を初期化
y = np.zeros((2, len(x_original)+ 2*N))   # 処理結果を納める配列を初期化 
for i in range(0, n_loop):
    buf = x[start : start + N]            # start からバッファの長さだけ切り出し
    buf = np.append(buf, np.zeros(N))     # 末尾に 0 を追加して (長さ: 2N)
    X   = np.fft.fft(buf)                 # FFT して
    for lr in LR:
        Y = H[LR[lr]] * X                 # スペクトルのかけ算して
        buf = np.real(np.fft.ifft(Y))     # 逆 FFT して
        y[LR[lr], start : start+2*N+N]   = overlap_add(y[LR[lr], start : start + 2*N], buf, N) 
        # この行：オーバーラップしながら書き戻す（長さがNだけ延びている）
    start += N                            # 次のバッファ区間の先頭に移動

times = np.arange( y.shape[1] ) * sampling_interval
for lr in LR:
    plt.title("Convolved music (" + lr + " ear)")
    plot_wave( times, y[LR[lr]] )


audio = Audio(y, rate = fs)
audio

(2) 5.4 節 (c) のバッファリングは，方形波窓の切り出しと同様な方式である。これを，窓長の半分ずつ推移するハニング窓を使ったバッファリングに書き直しなさい。（前章でみたとおり，「窓長の半分ずつ推移するハニング窓」は，一旦はある時刻の音を 2 つのブロックに分けるが，加算すれば元の瞬時値を復元できるので，5.4 節 (c) で処理したのと同じ出力を得ることができるはずである。）

【答】インプット5.21を書き替えてみました。要点は，切り出しの推移を窓長の半分ずつずらすので，オーバーラップアドするときにも，その分だけオーバーラップを多くすることがあることでしょう。

In [ ]:
# インプット 5.21
fs = 44.1e3
N = 1024               # フィルタのタップ長 と バッファ長
fc = 6e3 / (fs / 2.0)  # 遮断周波数 6 kHzを，ナイキスト周波数 (fs/2) に対する比で表現 

# --- フィルタの準備 ---
h = signal.firwin(N, fc, window='hann') # LPFの設計
plt.title("Output 5.21")
plot_wave([], h)
H = np.fft.fft(np.append(h, np.zeros(N)))

# --- 音の準備 ---
fs, x_original = scipy.io.wavfile.read ('sample/sample3.wav')
print('Sampling frequency =', fs, '[Hz]')
sampling_interval = 1.0 / fs
times = np.arange ( len ( x_original )) * sampling_interval
x = x_original.copy()

mod = 2e3*np.cos(2*np.pi*0.25*times)
noise = 500.0 * np.sin(2*np.pi*8e3*times + mod)
x += noise.astype(int)                           # 雑音を重畳

# --- 処理の開始 ---
shift = int(N/2)                         # ハニング窓のシフトは N/2
n_loop = int(np.floor(len(x) / shift))-1 # 最後の区間長に満たない点は，処理対象から除外

start = 0                                 # 切り出し区間の開始点を初期化
y = np.array([])                          # 処理結果を納める配列を初期化 
for i in range(0, n_loop):
    buf = hanning(N) * x[start : start + N]  # start からバッファの長さだけ切り出し，窓掛け
    buf = np.append(buf, np.zeros(N))     # 末尾に 0 を追加して
    X   = np.fft.fft(buf)                 # FFT して
    Y   = H * X
    buf = np.real(np.fft.ifft(Y))         # 逆 FFT して 
    y   = overlap_add(y, buf, N+shift)   # 結果をつなげる
    start += shift                       # 次のバッファ区間の先頭に移動

times = np.arange ( len( y )) * sampling_interval
plot_wave ( times, y )

plt.specgram(y, NFFT=2408, Fs=fs, cmap='jet')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.show()

audio = Audio(y, rate = fs)
audio

(3) 5.4節 (c) で紹介した方法は「重畳加算法（Overlap-add method)」と呼ばれる。同じ目的を達成する別の方法として「重畳保留法 (Overlap-save method)」がある。この方法について調べ，重畳加算法に対するメリットを述べなさい。

【答】重畳保留法を簡単に説明する。$h(n)$は$N$点とし，その後に$N$点の0を付加して$\hat h(n)$とするのは重畳加算法と同じである。重畳保留法では，入力$x(n)$を2$N$の長さでバッファする（最初の$N$点に関しては，その先頭に$N$個の0を付加することで2$N$点とする）。そして，2$N$点同士のFFTの積を逆FFTして時間波形に戻す。その前半$N$点は捨て，後半$N$点のみを出力とする方法である。オーバーラップの計算を省けるだけ，重畳加算法よりも計算コストは低いというメリットがある（本書では，「説明が容易そう」という理由で，重畳加算法を紹介した）。